In [1]:
run ../utils/preprocess.py

In [2]:
import constants as C

import matplotlib.pyplot as plt

In [ ]:
def tables_from_category(category):
    with open('../../data/pickle_files/%s.pickle' % category, 'rb') as f:
        return pickle.load(f)

qa_table, reviews_table = tables_from_category(C.VIDEO_GAMES)

#Question with maximum length

In [ ]:
mask = qa_table['questionText'].str.len() == 20
result = qa_table.loc[mask]['questionText']
question = result.iloc[1]

In [ ]:
print question

In [ ]:
stopset_u = set(stopwords.words('english'))
stopset = set([str(sw) for sw in stopset_u])

tokens = nltk.word_tokenize(question)
tokens = [w.lower() for w in tokens if not w.lower() in stopset]

print len(tokens)

#Questions containing "Read More"

In [ ]:
mask = qa_table['answerText'].str.contains("Read More")
result = qa_table.loc[mask]['questionText']

In [ ]:
#Preprocess

In [ ]:
reviewsLengthList, questionsLengthList, answersLengthList = generate_length_lists(C.VIDEO_GAMES)

In [ ]:
df = get_raw_dataframe(C.VIDEO_GAMES)

In [ ]:
len(df)

In [ ]:
item = C.LENGTH_DICT[16]
C.JSON_DATA_PATH = '../../data/json_data'
C.INPUT_DATA_PATH = '../../data/input'

In [ ]:
filter_raw_data(item['Category'], item['ReviewLength'], item['QuestionLength'], item['AnswerLength'])

In [ ]:
pd = get_filter_dataframe(C.VIDEO_GAMES)

In [ ]:
pd.head()

In [3]:
category = "Video_Games"
C.INPUT_DATA_PATH = '../../data/90_input'
with open('%s/train-%s.pickle' % (C.INPUT_DATA_PATH, category), 'rb') as f:
    train_data = pd.read_pickle(f)

In [4]:
train_data.head()

,asin,questionsList,reviewsList
89,B0018YXM3Y,[{'text': 'does this this game run on windows ...,"[{'unhelpful': 7, 'text': 'This is a great gam..."
114,B005761E70,[{'text': 'are the thumstick controls more pre...,"[{'unhelpful': 0, 'text': 'I got this controll..."
419,B00GHTP7SS,[{'text': 'This is only the code? Where do i d...,"[{'unhelpful': 0, 'text': 'the game runs well ..."
172,B001BZ2F56,[{'text': 'Is this software work well with Win...,"[{'unhelpful': 2, 'text': 'I really like tycoo..."
208,B0091WIOJ2,[{'text': 'Do i need a 7.1 Surround sound card...,"[{'unhelpful': 0, 'text': 'my review, my opini..."


In [6]:
for index, row in train_data.head().iterrows():
    print(index, row['asin'])

89 B0018YXM3Y
114 B005761E70
419 B00GHTP7SS
172 B001BZ2F56
208 B0091WIOJ2


In [ ]:
for row in train_data.head(2):
    print(row['questionsList'])
    for question in questionsList:
        for answer in question['answers']:
                print(answer['text'])

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

import numpy as np
from operator import itemgetter, attrgetter
from torch.utils.data import Dataset, DataLoader

In [ ]:
reviews1 = [[1,2], [1,2,3],[6,7,8]]
reviews2 = [[1], [1,2]]

question1 = [1,2,3]
question2 = [2,3,4,5]

answer1 = [1,3]
answer2 = [2]

data1 = [(answer1), (answer2)]
data2 = [(answer1, question1), (answer2, question2)]
data3 = [(answer1, question1, reviews1), (answer2, question2, reviews2)]

In [ ]:
print(data1)
print(data2)
print(data3)

In [ ]:
a, q = zip(*data2)
print(a)
print(q)

In [ ]:
class AmazonDataLoader(DataLoader):

    def sortByLength(self, item):
        if self.mode is "1":
            return len(item)
        
        elif self.mode is "2":
            assert(len(item) == 2)
            return len(item[0])
        
        elif self.mode is "3":
            assert(len(item) == 3)
            reviews = item[2]
            max_len = 0
            for review in reviews:
                max_len = max(max_len, len(review))
            return max_len

        
    def __init__(self, data, mode, batch_size):
        self.batch_size = batch_size
        self.mode = mode
        
        sorted(data, key=self.sortByLength, reverse=True)    
        self.data = data
        

    def create_packed_qa(self, batch_data):
        lengths = np.array([len(item) for item in batch_data])
        max_len = max(lengths)
        
        padded_data = np.array( [np.pad(item, (0, max_len-len(item)), 'constant') for item in batch_data] )
        padded_data = torch.from_numpy(padded_data)
        
        return (padded_data)
    
    def create_packed_reviews(self, review_data):
        max_num_reviews = 0
        for reviews in review_data:
            max_num_reviews = max(max_num_reviews, len(reviews))
        
        data = []
        for i in range(max_num_reviews):
            batch_data = []
            for j in range(self.batch_size):
                reviews = review_data[j]
                if i < len(reviews):
                    batch_data.append(reviews[i])
                else:
                    batch_data.append([0])
            data.append(batch_data)
        
        padded_data = []
        for i in range(max_num_reviews):
            batch_data = data[i]
            lengths = [len(review) for review in batch_data]
            max_len = max(lengths)
            
            padded_batch_data = np.array([np.pad(item, (0, max_len-len(item)), 'constant') for item in batch_data])
            padded_batch_data = torch.from_numpy(padded_batch_data)
            
            padded_data.append(padded_batch_data)
            
        return padded_data
        
    def __iter__(self):
        print(self.data)
        self.num_batches = len(self.data) // self.batch_size
        indices = np.arange(self.num_batches)
        np.random.shuffle(indices)

        for index in indices:
            start = index*self.batch_size
            end = (index+1)*self.batch_size
            
            batch_data = self.data[start:end]
            assert(self.batch_size == len(batch_data))
            
            if self.mode is "1":
                answers = batch_data
                packed_answers = self.create_packed_qa(answers)
                yield (packed_answers)
                
            elif self.mode is "2":
                answers, questions = zip(*batch_data)
                packed_answers = self.create_packed_qa(list(answers))
                packed_questions = self.create_packed_qa(list(questions))
                yield (packed_answers, packed_questions)
            
            elif self.mode is "3":
                answers, questions, reviews = zip(*batch_data)
                packed_answers = self.create_packed_qa(list(answers))
                packed_questions = self.create_packed_qa(list(questions))
                packed_reviews = self.create_packed_reviews(list(reviews))
                yield (packed_answers, packed_questions, packed_reviews)
                
    def __len__(self):
        return self.num_batches

In [ ]:
data_loader = AmazonDataLoader(data3, "3", 2)

In [ ]:
for batch_idx, data in enumerate(data_loader):
    answers, questions, reviews = data
    print(data)

In [ ]:
def sortByMaxReviewLength(item):
    reviews = item[2]
    max_len = 0
    for review in reviews:
        max_len = max(max_len, review.shape[0])
    return max_len

for item in data3:
    assert(len(item) == 3)

sorted(data3, key=sortByMaxReviewLength, reverse=True)

In [ ]:
def sortByAnswerLength(item):
    return item[0].shape[0]

sorted(data2, key=sortByAnswerLength, reverse=True)

In [ ]:
answers = data1
print(answers)

answers, questions = zip(*data2)
print(data2)
print(answers)
print(questions)

answers, questions, reviews = zip(*data3)
print(data3)
print(answers)
print(questions)
print(reviews)

In [ ]:
x = Variable(torch.randn(10, 10, 30))
lens = list(range(1, 11))
x = pack_padded_sequence(x, lens[::-1], batch_first=True)
y = pad_packed_sequence(x)